Скачиваю файлы

## Библиотеки и настройки

In [1]:
#Загружаю нужные библиотеки

import pandas as pd
import math

import datetime

# регулярки
import re


# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [3]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [4]:
# путь к файлам

file_path = '/'
proxy_list_path = 'C:/work/keys/proxy_list.txt' # путь и названия со списком прокси

In [5]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

Рабочих прокси — 9


In [6]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

Номер прокси, от 0 до 8
1


In [7]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
# opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [8]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

Login: borisgimatij0Hhgp
Pass: 9YQqEGNsKE


In [9]:
# запускаю драйвер, проверяю настройки

#s=Service('C:/Users/bgi/Anaconda3/chromedriver.exe')
#driver = webdriver.Chrome(service=s, options=opt)

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки

Пагинация простая, номер страницы на конце урла.

In [10]:
# ссылка на 

url_main = 'https://fsin-atlas.ru/?page='

In [11]:
# открываю страницу

driver.get(url_main + '1')
sleep(3)

In [12]:
driver.find_element(By.CLASS_NAME, 'paginator').text

'страница 1 из 31 (924 учреждения) Следующая'

In [13]:
# основной скрипт

data_url = []

for i in tqdm(range(1, 32)): # цикл для перехода по страницам
    
    driver.get(url_main + str(i))
    sleep(3)
    
    # блоки учреждений
    prison_blocks = driver.find_elements(By.CLASS_NAME, 'catalog_item')
    
    for prison_block in prison_blocks: # цикл для извлечения данных
        
        prison_name = prison_block.find_element(By.CLASS_NAME, 'catalog_item_title').text # название
        prison_url = prison_block.find_element(By.CLASS_NAME, 'catalog_item_title').get_attribute('href') # урл заведения
        prison_info = prison_block.find_element(By.CLASS_NAME, 'catalog_item_params').text.split('\n') # текстовый блок
        prison_address = prison_info[0] # адрес
        reviews = prison_info[1] # кол-во комментариев
        views = prison_info[2] # кол-во просмотров
        stars = prison_block.find_element(By.CLASS_NAME, 'stars_wrapper').text # оценка
        
        data_url.append([prison_name, prison_url, prison_address, reviews, views, stars, prison_info])

  0%|          | 0/31 [00:00<?, ?it/s]

In [14]:
head = ['title', 'prison_url', 'address', 'reviews', 'views', 'stars', 'all_info']
work_urls = pd.DataFrame(data_url, columns=head)

In [15]:
work_urls.sample()

,title,prison_url,address,reviews,views,stars,all_info
667,КП-6 Калуга,https://fsin-atlas.ru/catalog/object/kp6kaluga/,"г. Калуга, ул. Николо- Козинская д.129",6 комментариев от посетителей,16171 просмотр учреждения,0.0,"[г. Калуга, ул. Николо- Козинская д.129, 6 ком..."


## Собираю данные со страниц учреждений

In [16]:
data_prison = [] # список для данных


for prison_url in tqdm(work_urls['prison_url'].to_list()): # цикл по ссылкам
    
    driver.get(prison_url) # открываю страницу
    sleep(3)
    
    all_lines = driver.find_element(By.ID, 'sizo_data').find_elements(By.TAG_NAME, 'tr') # забираю таблицу с описанием
    
    description_dict = {}
    
    for one_line in all_lines: # разбираю строчку
        two_columns = one_line.find_elements(By.TAG_NAME, 'td')
        
        temp_list = [] # временный список

        for one_column in two_columns: #
            temp_list.append(one_column.text)

        description_dict[temp_list[0]] = temp_list[1]

    region = description_dict['Расположение:']
    department = description_dict['УФСИН/ГУФСИН:']
    full_address = description_dict['Полный адрес:']
    description = description_dict['Описание:']
    places = description_dict['Лимит наполнения:']
    teamlead = description_dict['Начальник учреждения:']
    contacts = description_dict['Контакты:']
    interest = description_dict['Популярность:']

    data_prison.append([prison_url, region, department, full_address, description, places, teamlead, contacts, interest])

  0%|          | 0/924 [00:00<?, ?it/s]

In [17]:
head = ['prison_url', 'region', 'department', 'full_address',
        'description', 'places', 'teamlead', 'contacts', 'interest']

work_prison = pd.DataFrame(data_prison, columns=head)

In [21]:
driver.quit()

In [19]:
work_prison.shape

(924, 9)

In [20]:
work_prison.sample(3)

,prison_url,region,department,full_address,description,places,teamlead,contacts,interest
311,https://fsin-atlas.ru/catalog/object/ik12kuibi...,"Новосибирская область, г. Куйбышев",ГУФСИН по Новосибирской области,"632387, , Новосибирская область, г. Куйбышев\n...",Мужская исправительная колония строгого режима,"1100 мест, включая участок колонии-поселения н...",подполковник внутренней службы Бехтин Евгений ...,"8 (38362) 53-419, 8 (383-62) 23-365\nik12@54.f...","Просмотры: 26128, комментарии: 13"
551,https://fsin-atlas.ru/catalog/object/ik29kamen/,"Приморский край, г. Большой Камень",ГУФСИН по Приморскому краю,"692806, ул. Дзержинского д. 1, Приморский край...",Мужская исправительная колония общего режима,1025 мест и пфрси на 165 мест,подполковник внутренней службы Пьянков Алексей...,8 (42335) 5-51-95\nkp-49@25.fsin.gov.ru\nwww.2...,"Просмотры: 18914, комментарии: 12"
204,https://fsin-atlas.ru/catalog/object/ik2talitsy/,"Ивановская область, пос. Талицы",УФСИН по Ивановской области,"155644, ул. Ленина д.18, Ивановская область, п...",Мужская исправительная колония строгого режима,1501 место,Врио начальника подполковник внутренней службы...,+7 (49347) 2-44-80\nik-2@37.fsin.gov.ru\nwww.3...,"Просмотры: 32577, комментарии: 15"


In [23]:
work_prison = work_prison.merge(work_urls, how='left', on='prison_url')

In [26]:
# Записываю в файл

work_prison.to_excel('C:/00_Data/prisons_2022_13_07.xlsx', encoding='utf-8', index=False)